To implement the encodeRS(m) function for a [255, 128] Reed Solomon code with generator polynomial g(x) = (x-1)(x-α)(x-α^2)*(x-α^3)...(x-α^127), we can use the following steps:

    Convert the input message m into a polynomial m(x) of degree k-1, where k = 128, by setting m(x) = m_0 + m_1x + ... + m_k-1x^(k-1).

    Pad the message polynomial m(x) with k = 128 zeros to form a new polynomial M(x) of degree n-1 = 254, where n = 255. That is, set M(x) = m(x)*x^k.

    Calculate the remainder polynomial R(x) = M(x) mod g(x) using polynomial long division or the Euclidean algorithm in GF(2^8).

    The codeword c can be formed by appending the coefficients of the remainder polynomial R(x) to the message polynomial m(x), i.e., set c(x) = m(x) + R(x), and then evaluate the resulting polynomial c(x) at each element of GF(2^8) to obtain the codeword c = (c_0, c_1, ..., c_n-1).

In [31]:
# d = floor(128-1 / 2) = 63 
# i.e correct up to 63 errors 


In [32]:
import secrets
q = 2 
m = 8 
p = x^8 + x^4 + x^3 + x^2 + 1
F.<a> = GF(q^m, modulus=p) 
Fqm.<y> = GF(q^m) 
R.<x> = PolynomialRing(F) 

n = 255
k = 128 

message_length = 128 # bytes 

In [33]:
def text2bin(msg):
    return "".join([f"{ord(m):08b}" for m in msg])

def bin2text(bin_s):
    return "".join([chr(int(bin_s[i: i+8], 2)) for i in range(0, len(bin_s), 8)])

def to_field_element(string):
    """Binary String to field Element"""
    elem = 0
    n = len(string)
    for i in range(n):
        coeff = int(string[n-i-1])
        elem += coeff * a^i # coeff is 0 or 1
    return elem


def from_field_element(field_element):
    """Field Element to binary sring"""
    return ''.join(map(str, vector(field_element)[::-1]))

def convert_str_to_polynomial(msg): 
    msg = msg + "x"*(k - len(msg)) 
    # generate random message 
    # message = [random.randint(0, 255) for _ in range(message_length)]
    message = text2bin(msg)
    message = [message[i:i+8] for i in range(0, k, 8)]
    
    message = sum([to_field_element(e)*x^i for i,e in enumerate(message)])
    
    return message

In [71]:
# define generator poly 
g_x = [(x - (a^i)) for i in range(q^m / 2 - 1)] 
# should I sum it ? 
# degree of generator poly. should be le #codewords
g = prod(g_x)

In [75]:
# sanity check 
C = codes.ReedSolomonCode(GF(q^m, 'a'), n, k)

In [76]:
Ccyc = codes.CyclicCode(code=C) 

In [77]:
true_g = Ccyc.generator_polynomial()

In [81]:
g

x^127 + (a^7 + a^5 + a^4)*x^126 + (a^6 + a^2 + a)*x^125 + (a^6 + a^5 + a^4 + a^2 + 1)*x^124 + (a^3 + a^2 + 1)*x^123 + (a^7 + a^4 + a)*x^122 + (a^6 + a^5 + a^3 + a + 1)*x^121 + (a^6 + a^5 + a^4 + a^2 + 1)*x^120 + (a^6 + a^4 + a^3 + a + 1)*x^119 + (a^6 + a^5 + a^3 + a^2 + 1)*x^118 + (a^6 + a^5)*x^117 + (a^6 + a^5 + a^4 + a^3 + a + 1)*x^116 + (a^5 + a^2 + a)*x^115 + (a^7 + a^5 + a^4 + a + 1)*x^114 + (a^6 + a^5 + a^2 + a)*x^113 + (a^7 + a^6 + a^2)*x^112 + (a^7 + a^5 + a^4 + a^3 + 1)*x^111 + (a^7 + a^3 + a^2 + a)*x^110 + (a^7 + a^6 + a^4 + a^2 + 1)*x^109 + (a^5 + a^4 + a^3 + 1)*x^108 + (a^6 + a^3 + a)*x^107 + (a^4 + a^3 + a)*x^106 + (a^5 + a^4 + a^2 + a + 1)*x^105 + a^4*x^104 + (a^7 + a^5)*x^103 + (a^6 + a^3 + 1)*x^102 + (a^4 + a^3 + a^2)*x^101 + (a^7 + a^6 + a^5 + a^2 + 1)*x^100 + a^7*x^99 + (a^7 + a^6 + a^5 + a^3 + a^2)*x^98 + (a^4 + a)*x^97 + (a^7 + a^5 + a^3)*x^96 + (a^6 + a^3 + a^2 + a + 1)*x^95 + (a^7 + a^3 + 1)*x^94 + (a^7 + a^6 + a^5 + a^4 + a^2 + a)*x^93 + (a^6 + a^5 + a^4 + a^3)*x

In [52]:

def encodeRS(g, m): 
    """
    Takes a 128 - byte messagee, and encode it to a codeword 
        c, with a systematic encoding as 
        
        m = (m_0,m_1,...,m_k-1) => c = (m_0, m_1,...,m_k-1, c_k,..._c_n-1)
    """
    m_x = convert_str_to_polynomial(m)
    print(m_x) 
    # Now perform encoding 
    # c(x) => m(x) * x^(n-k) % g(x) 
    # formula book : 
    # m(x)*x^(n-k) - Remainder(m(x)*x^(n-k), g(x)) 
    
    tmp = m_x * x^(n-k) 
    c_x = tmp % g
    # _, rem = tmp.quo_rem(g)
    # c_x = tmp - rem
    # c_x = c_x
    return c_x



In [53]:
text = "This is a sample text, that is not too long"  

r = encodeRS(g_x, text)
print() 
print(r) 


(a^6 + a^5 + a^2 + 1)*x^15 + (a^6 + a^5 + a^3 + a^2)*x^14 + (a^6 + a^5 + a^4)*x^13 + (a^6 + a^5 + a^3 + a^2 + 1)*x^12 + (a^6 + a^5 + 1)*x^11 + (a^6 + a^5 + a^4 + a + 1)*x^10 + a^5*x^9 + (a^6 + a^5 + 1)*x^8 + a^5*x^7 + (a^6 + a^5 + a^4 + a + 1)*x^6 + (a^6 + a^5 + a^3 + 1)*x^5 + a^5*x^4 + (a^6 + a^5 + a^4 + a + 1)*x^3 + (a^6 + a^5 + a^3 + 1)*x^2 + (a^6 + a^5 + a^3)*x + a^6 + a^4 + a^2


TypeError: no common canonical parent for objects with parents: 'Univariate Polynomial Ring in x over Finite Field in a of size 2^8' and '<class 'list'>'

In [28]:

# Decoding part 

def decodeRS(r): 
    
    # calulate syndroms 
    t = floor((k - 1) / 2)
    syndroms = [r(a^i) for i in range(1, t + 1)]
    for _ in syndroms: 
        print(_)
    
    
decodeRS(r)

a^6 + a^5 + a^4 + a^3 + a^2 + 1
a^6 + a^5 + a^4 + a^3 + a^2 + 1
a^7 + a^5 + a^4 + a^2
a^7 + a^6 + a^5 + a^3 + a^2
a^5 + a^4 + a^3 + a^2 + a
a^6 + a^5 + a
a^6 + a^3 + a^2 + a
a^4 + a + 1
a^7 + a^3 + a
a^5 + a^4 + a^3
a^5 + a^4 + a^3
a^7 + a^4 + 1
a^7 + a^5
a^6 + a^4 + a^3 + a^2 + 1
a^4 + a^3 + a^2 + a + 1
a^7 + a^6 + a^4 + a^3 + a^2 + 1
0
a^7 + a^6 + a^5 + a^4 + a^2 + a + 1
a^4 + a^3 + a + 1
a^7 + a^6 + a^5 + a^4 + a^3 + a^2
a^5 + a + 1
a^7 + a^6 + a
a^7 + a^6 + a^5 + a^4 + a^3 + 1
a^6 + a^5 + a^4 + a^3 + a^2
a^4 + a^2
a^7 + a^6 + a^5 + a^4 + a^3 + 1
a^5 + a^3 + a^2 + a + 1
a^7 + a^5 + a^4 + a^3 + a^2 + a
a^5 + a^3 + a^2 + 1
a^7 + a^6 + a^5 + a^4 + a^2 + a
a^6 + a^5 + a^4 + a + 1
a^6 + a^5 + a^4 + a^2 + a
a^6 + a^4 + a^2 + 1
a^7 + a^5 + a^3 + a^2 + a
a^7 + a^2 + a
a^7 + a^6 + a^3 + a
a^4 + a^3 + a^2 + a + 1
a^7 + a^6 + a^5 + a^4 + a^2 + 1
a^5
1
a^4
a^7 + a^6 + a^4 + a^3 + a^2
a^2 + a + 1
a^7 + a^5 + a^4 + a + 1
a^4 + a + 1
a^7 + a^4
a^6 + a^4 + a^3 + a^2 + a
a^5 + a^3 + 1
a^7 + a^5 + a^